In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')
#data_folder = Path('E:/Downloads/')
#data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')

df = pd.read_csv(data_folder/'train2_irf_00006.csv', \
                 dtype='int8')
y = pd.read_csv(data_folder/'y2.csv', \
                sep=';', usecols=[1], dtype='int8')

print(df.shape, y.shape)

(26495, 4439) (26495, 1)


In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt import space_eval
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=PendingDeprecationWarning)

def Performance(Model,Y,X):
    AUC = roc_auc_score(Y, Model.predict_proba(X)[:,1])
    aucs.append(AUC)
    print ('the AUC is : %0.4f' %  AUC)

def auc_model(params):
    clf = LogisticRegression(**params)
    return cross_val_score(clf, train_x, train_y, scoring='roc_auc').mean()

def last_model(params):
    clf = LogisticRegression(**params)
    clf.fit(s1_x, s1_y)
    Performance(clf, valid_y, valid_x)
    return clf

param_space = {
    'C': hp.choice('C', [0.001,0.01,0.1,1,10,100]),
    'penalty': hp.choice('max_features', ['l1','l2']),
}

def f(params):
    global best
    global best_params
    auc = auc_model(params)
    if auc > best:
        best = auc
        best_params = params
        #print ('new best:', best, params)
    return {'loss': -auc, 'status': STATUS_OK}

aucs = []

s1 = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
for t1_index, val_index in s1.split(df, y):
    s1_x, s1_y = pd.DataFrame(df.iloc[t1_index]), y.iloc[t1_index].values.ravel()
    valid_x, valid_y = df.iloc[val_index], y.iloc[val_index].values.ravel()
 
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
for train_index, test_index in cv.split(s1_x, s1_y):
    train_x, train_y = pd.DataFrame(s1_x.iloc[train_index]), s1_y[train_index]
    test_x, test_y = s1_x.iloc[test_index], s1_y[test_index]

    trials = Trials()
    best = 0
    best = fmin(f, param_space, algo=tpe.suggest, max_evals=50, trials=trials)
    print ('\nbest:')
    last_model(best_params)
    print (space_eval(param_space, best))
    print('\n')
print(np.mean(aucs))